In [1]:
# コメント文の分散表現をDoc2Vecを用いて作る。
# 副生産物としてコンテキストの分散表現を得る。

from gensim.models import Word2Vec
from tensorflow import keras
import tensorflow as tf
from keras.models import load_model
import time
import glob
import os
import os.path
import random
import sys
import numpy as np
import nltk
nltk.download('punkt')

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop

sys.path.append('..')
sys.path.append('../..')

Using TensorFlow backend.
[nltk_data] Downloading package punkt to /Users/makoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
file = glob.glob("../../repositories_cleansing/repositories2TokenWithCommentDownOnlyRawDataWithIdent20Tokens/all.txt")
print(file)

['../../repositories_cleansing/repositories2TokenWithCommentDownOnlyRawDataWithIdent20Tokens/all.txt']


In [3]:
#トークンのサイズ
tokenSize=20

#コメントのベクトルのサイズ
commentVecSize=100

# コンテキストのベクトルサイズ
contextVecSize = 100

In [4]:
comments = []
contexts = []
f = open(file[0])
lines = f.readlines()

i=0
for line in lines:
    temp_data = line.replace("\n","")
    temp_context = " ".join(temp_data.split(' ')[0:tokenSize])
    temp_comment = " ".join(temp_data.split(' ')[tokenSize:])
    morphComment = nltk.word_tokenize(temp_comment)
    morphContext = nltk.word_tokenize(temp_context)
    contexts.append(TaggedDocument(morphContext,[i]))
    comments.append(TaggedDocument(morphComment,[i]))
    i += 1
f.close()


In [5]:
print(len(comments))
print(len(contexts))

print(contexts[0])
print()
print(comments[0])

48920
48920
TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [0])

TaggedDocument(['#', '!', '/usr/bin/env', 'ruby', '#', '--', '#', 'Copyright', '2004', 'Austin', 'Ziegler', '<', 'ruby-install', '@', 'halostatue.ca', '>', '#', 'Install', 'utility', '.', 'Based', 'on', 'the', 'original', 'installation', 'script', 'for', 'rdoc', 'by', 'the', '#', 'Pragmatic', 'Programmers', '.', '#', '#', 'This', 'program', 'is', 'free', 'software', '.', 'It', 'may', 'be', 'redistributed', 'and/or', 'modified', 'under', '#', 'the', 'terms', 'of', 'the', 'GPL', 'version', '2', '(', 'or', 'later', ')', 'or', 'the', 'Ruby', 'licence', '.', '#', '#', 'Usage', '#', '--', '--', '-', '#', 'In', 'most', 'cases', ',', 'if', 'you', 'have', 'a', 'typical', 'project', 'layout', ',', 'you', 'will', 'need', 'to', 'do', '#', 'absolutely',

In [6]:
commentModel = Doc2Vec(documents=comments, vector_size=commentVecSize,min_count=3,window=5,epochs=20,dm=0)

contextModel = Doc2Vec(documents=contexts, vector_size=contextVecSize,min_count=3,window=5,epochs=20,dm=0)

In [7]:
print(comments[0])
print(commentModel.docvecs[0])

TaggedDocument(['#', '!', '/usr/bin/env', 'ruby', '#', '--', '#', 'Copyright', '2004', 'Austin', 'Ziegler', '<', 'ruby-install', '@', 'halostatue.ca', '>', '#', 'Install', 'utility', '.', 'Based', 'on', 'the', 'original', 'installation', 'script', 'for', 'rdoc', 'by', 'the', '#', 'Pragmatic', 'Programmers', '.', '#', '#', 'This', 'program', 'is', 'free', 'software', '.', 'It', 'may', 'be', 'redistributed', 'and/or', 'modified', 'under', '#', 'the', 'terms', 'of', 'the', 'GPL', 'version', '2', '(', 'or', 'later', ')', 'or', 'the', 'Ruby', 'licence', '.', '#', '#', 'Usage', '#', '--', '--', '-', '#', 'In', 'most', 'cases', ',', 'if', 'you', 'have', 'a', 'typical', 'project', 'layout', ',', 'you', 'will', 'need', 'to', 'do', '#', 'absolutely', 'nothing', 'to', 'make', 'this', 'work', 'for', 'you', '.', 'This', 'layout', 'is', ':', '#', '#', 'bin/', '#', 'executable', 'files', '--', '``', 'commands', "''", '#', 'lib/', '#', 'the', 'source', 'of', 'the', 'library', '#', '#', 'The', 'default

In [8]:
# 文章IDが0の文章と似た文章とその内積を得ることが出来る。
commentModel.docvecs.most_similar(0)

[(48893, 0.5595793128013611),
 (47295, 0.5578314065933228),
 (26200, 0.557135820388794),
 (10013, 0.5496345162391663),
 (6831, 0.5489787459373474),
 (45645, 0.5414491891860962),
 (5773, 0.5412502288818359),
 (12980, 0.5406416058540344),
 (31088, 0.5391349196434021),
 (21598, 0.5389913320541382)]

In [9]:
print(comments[48893])
print(comments[6831])
print(comments[47295])

TaggedDocument(['#', 'Cleanup', 'old', 'assets', 'in', 'the', 'compile', 'directory', '.', 'By', 'default', 'it', 'will', '#', 'keep', 'the', 'latest', 'version', ',', '2', 'backups', 'created', 'within', 'the', 'past', 'hour', '.', '#', '#', 'Examples', '#', '#', 'To', 'force', 'only', '1', 'backup', 'to', 'be', 'kept', ',', 'set', 'count=1', 'and', 'age=0', '.', '#', '#', 'To', 'only', 'keep', 'files', 'created', 'within', 'the', 'last', '10', 'minutes', ',', 'set', 'count=0', 'and', '#', 'age=600', '.', '#'], [48893])
TaggedDocument(['#', 'As', 'per', 'the', 'BSD', 'manual', 'page', ',', 'set', 'if', 'this', 'is', 'a', 'directory', ',', 'or', 'if', '#', 'any', 'execute', 'bit', 'is', 'set', 'on', 'the', 'original', '(', 'unmodified', ')', 'mode', '.', '#', 'Ignored', 'otherwise', ';', 'it', 'is', '``', 'add', 'if', "''", ',', 'not', '``', 'add', 'or', 'clear', "''", '.'], [6831])
TaggedDocument(['#', 'Batch', 'file', 'deletions', ',', 'deleting', 'over', '130,000+', 'files', 'will',

In [10]:
# 文章IDが3の文章と似た文章とその内積を得ることが出来る。
print(comments[3])
commentModel.docvecs.most_similar(3)

TaggedDocument(['#', 'to_f', 'is', "n't", 'quite', 'exact', 'for', 'strings', 'like', '``', '1.5.1', "''", 'but', 'is', 'good', '#', 'enough', 'for', 'this', 'purpose', '.'], [3])


[(46428, 0.8304678797721863),
 (10579, 0.8175169825553894),
 (10683, 0.8035566210746765),
 (42715, 0.8013337850570679),
 (20442, 0.795933723449707),
 (21750, 0.795441746711731),
 (20965, 0.7908487915992737),
 (11791, 0.7889671325683594),
 (25185, 0.7867152094841003),
 (9431, 0.786310613155365)]

In [11]:
print(comments[21156])
print(comments[46428])
print(comments[8996])

TaggedDocument(['#', 'only', 'consume', 'for', '``', 'k4', "''"], [21156])
TaggedDocument(['#', 'Style/LambdaCall', 'checks', 'for', 'this', 'syntax'], [46428])
TaggedDocument(['#', 'OpenBSD', 'uses', 'a', 'format', 'like', '``', 'january', '1', '1970', "''"], [8996])


In [12]:
commentModel.wv.most_similar(positive=['if', 'while'], negative=['for'])

[('Metrics/CyclomaticComplexity', 0.4190030097961426),
 ('Actor', 0.3959883451461792),
 ('ten', 0.35167044401168823),
 ('c', 0.3442232608795166),
 ('running', 0.3435203433036804),
 ('1111', 0.326791375875473),
 ('dispatched', 0.3259218633174896),
 ('sublocated', 0.3259182572364807),
 ('cleanup', 0.3207174241542816),
 ('warn', 0.30853238701820374)]

In [13]:
print(contexts[0])
print(contextModel.docvecs[0])

TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [0])
[ 0.14942591  0.04476078 -0.08624335  0.20802586 -0.16894369  0.01417536
  0.12015094 -0.27508286 -0.00056314  0.06796762  0.26169962 -0.17186958
 -0.03121093  0.15890312 -0.06047279 -0.15954329  0.18595025  0.0173465
  0.08395198  0.10360787  0.02072055  0.13271223  0.04694368  0.2899401
  0.03773314  0.19109818 -0.07554463  0.17902733 -0.17628229  0.10071837
  0.0534431   0.05054125  0.0134237   0.07988356  0.03521914 -0.08090281
 -0.04948153 -0.01012222  0.2610698  -0.01909072  0.08893159  0.05091891
 -0.08425293  0.07588919  0.18043168 -0.10571592 -0.2595168   0.04802063
  0.05657126  0.14225365 -0.25362933  0.08394774  0.05786087 -0.05642432
 -0.18143529  0.34933048  0.35458565  0.2542341  -0.06130609 -0.1409034
 -0.02983069  0.29059738 -0.04218122

In [14]:
# 文章IDが0の文章と似た文章とその内積を得ることが出来る。
contextModel.docvecs.most_similar(0)

[(2924, 0.9926709532737732),
 (1395, 0.9916930794715881),
 (10083, 0.9910852909088135),
 (10655, 0.9907947778701782),
 (46648, 0.9902456402778625),
 (10659, 0.9893110394477844),
 (10656, 0.9891499280929565),
 (22872, 0.9889997243881226),
 (10749, 0.9886319637298584),
 (34952, 0.9884748458862305)]

In [15]:
print(contexts[6584])
print(contexts[10656])
print(contexts[45548])

TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [6584])
TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [10656])
TaggedDocument(['base_branch', '=', 'self', '.', 'pr_json', '[', '``', 'on_tstring_content', '``', ']', '[', '``', 'on_tstring_content', '``', ']', 'base_commit', '=', 'self', '.', 'pr_json'], [45548])


In [16]:
# 文章IDが3の文章と似た文章とその内積を得ることが出来る。
print(contexts[3])
contextModel.docvecs.most_similar(3)

TaggedDocument(['UNK', '=', 'facter', '.', 'version', '.', 'to_f', 'if', 'UNK', '<', 'UNK', 'puts', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'UNK'], [3])


[(9054, 0.9159144163131714),
 (6871, 0.9131546020507812),
 (9055, 0.9129316806793213),
 (2416, 0.9113594889640808),
 (11775, 0.9095793962478638),
 (2708, 0.9080649018287659),
 (43559, 0.9064557552337646),
 (41762, 0.9055429100990295),
 (34035, 0.9048767685890198),
 (30968, 0.9040793180465698)]

In [17]:
print(contexts[9055])
print(contexts[9054])
print(contexts[11303])

TaggedDocument(['if', 'resource_name', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'resource_name', '}', '``', 'else', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{'], [9055])
TaggedDocument(['UNK', '<', '<', 'UNK', 'if', 'resource_name', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'resource_name', '}', '``', 'else', 'UNK', '<', '<'], [9054])
TaggedDocument(['installed_range', '=', '``', 'on_tstring_content', '#', '{', 'version', '}', 'on_tstring_content', '#', '{', 'version', '.', 'major', '}', 'on_tstring_content', '``', 'graph', '.', 'UNK', '(', "'"], [11303])


In [18]:
contextModel.save("context2VecDoc2Vec.model")

# commentModel.save("comment2VecDoc2Vec.model")

In [19]:
contextModel.wv.most_similar(positive=['if', 'while'], negative=['for'])

[('c', 0.3442232608795166),
 ('running', 0.3435203433036804),
 ('element_matches', 0.33222806453704834),
 ('clazz', 0.3300752639770508),
 ('new_setting', 0.3220473527908325),
 ('cleanup', 0.3207174241542816),
 ('page_number', 0.31760305166244507),
 ('load_resources', 0.3168272078037262),
 ('tick', 0.31372538208961487),
 ('warn', 0.30853238701820374)]